This notebook is used to create a csv file that includes my point projections in a format similar to fplreview.com. This csv file can then be used for making advanced team optimization plans using Sertalp B. Cay's repo "fpl-optimization".

In [ ]:
latest_gameweek = 29

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import json

# Save team data

In [ ]:
# get login credentials for fetching team data
file_path = Path('../../login_credentials/fpl_login.json')
login_credentials = pd.read_json(file_path, typ='series')

In [ ]:
# utility function for fetching team data
from src.utils import fetch_my_team

In [ ]:
# fetch my team data
my_team = fetch_my_team(login_credentials.user_name, login_credentials.password, login_credentials.team_id)

In [ ]:
# save team data
file_path = Path('../data/team.json')
with open(file_path, 'w') as f:
    json.dump(my_team, f)

# Edit projections into fplreview format 
### (to be used with fpl-optimization repo solver)

In [ ]:
filepath = Path(f'../data/predictions/gameweek{latest_gameweek}.csv')
projections = pd.read_csv(filepath, index_col=0)
display(projections.head())
display(projections.shape)

In [ ]:
filepath = Path('../data/fpl_df.csv')
fpl_df = pd.read_csv(filepath, index_col=0, low_memory=False)
fpl_df = fpl_df[fpl_df.season=='23-24']
display(fpl_df.head())
display(fpl_df.shape)

In [ ]:
df = fpl_df.groupby('name').last().reset_index()[['id', 'name','points_per_game', 'total_points',]]
#df['id'] = df.id.astype(int)
df['games_played'] = np.round(np.where(df['points_per_game']!=0, df['total_points'] / df['points_per_game'], 0),0)
display(df.head())
display(df.shape)

In [ ]:
# drop duplicate players (some players get new spelling for their name during the season causing duplicates)
duplicate_ids = df.loc[df.id.duplicated(), 'id'].unique()
for id in duplicate_ids:
    ix = df.loc[df.id==id, 'games_played'].idxmin()
    df = df.drop(ix)
display(df.head())
display(df.shape)

In [ ]:
# drop unneccesary columns
df = df.drop(['points_per_game', 'total_points','games_played'], axis=1)
display(df.head())
display(df.shape)

In [ ]:
# merge id info to projections
projections = projections.merge(df, on='name', how='left')
#projections.head()

In [ ]:
# add xMins variable that is needed later
projections['xMins'] = 90

In [ ]:
projections[['id', 'expected_points', 'xMins', 'gameweek']]

In [ ]:
projections_pivot = (
    projections
    .pivot_table(
    columns=['gameweek',],
    index='id', 
    values=['expected_points','xMins'], 
    aggfunc='sum'
    )
)

projections_pivot.head()

In [ ]:
new_cols = []
for col in projections_pivot.columns:
    if col[0] == 'expected_points':
        new_col = str(col[1]) + '_Pts'
        new_cols.append(new_col)
    elif col[0] == 'xMins':
        new_col = str(col[1]) + '_xMins'
        new_cols.append(new_col)

projections_pivot.columns = new_cols
projections_pivot = projections_pivot.reset_index()
projections_pivot = projections_pivot.rename(columns={'id':'ID'})

projections_pivot.head()

In [ ]:
filepath = Path('../data/projections_pivot_table.csv')
projections_pivot.to_csv(filepath)

In [ ]:
projections_pivot[projections_pivot.ID==19]